In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from bidding.operations import create_bidding_app, setup_bidding_app, place_bid, cancel_bid, place_accept, close_bidding
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 1906
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


In [ ]:
#n_pri_key, n_address = generate_rekeyed_algorand_keypair(client, bidder)
# n_address = 'FOWXDCFQJKY3ISR7A5ZC2K5QEA5EYOJYCML55JWZVZYWWKQSUD6UFWIWKU'
# set_rekeyed_address(bidder.get_address(), n_address, 1)

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
storeApp.create_app()
storeAppID = storeApp.app_id

# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction OEB7RVGXB7AGULY6TUI7YGCZI5TEIZ4FCSM7QYH2ESFCBMJEFNOQ confirmed in round 110686.
Store App ID: 2185
Store App address: CQMHKYSIKGNJMFOWN26IONXHWVY6FJ6LZKOKFTVMHSPSSKCE7DDCZR7GSI
Waiting for confirmation...
Transaction IHSEGKYAZGODZRLC3L7KKIEN5UYOAX6V3LXSFM2V6WUHBCU7WPQQ confirmed in round 110688.
Store App Address: CQMHKYSIKGNJMFOWN26IONXHWVY6FJ6LZKOKFTVMHSPSSKCE7DDCZR7GSI


In [7]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 2UFYHOXESX5XPKKXEQZ6U64DIF4CUBCMIUFVMPSS53WW7RKIEGOA confirmed in round 110695.
Waiting for confirmation...
Transaction 4777OMFG6QFYFWCYPYDMA65ADZISLW4VI7WWSNETZY2LJS2VOHCQ confirmed in round 110697.
App ID: 2187
App Address: S3TPJY4OYJSSKPTHDVLRYTGSHCGVCPFFQAUZDA6SRIHJZZPJ6BHNJUISAU


Alice is setting up and funding token auction...

In [ ]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

In [ ]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

In [ ]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

In [ ]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

In [ ]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [ ]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [ ]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

In [ ]:
place_accept(client, creator, appID, seller, bidder.get_address(), bid_index)

Alice is closing out the bidding contract....

In [ ]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))